In [7]:
import cv2
import mediapipe as mp
import pandas as pd
from IPython.display import display, clear_output, Image

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture("../data/yulia_sochi_free.mp4")

# Create an empty list to store the landmarks data
landmarks_data = []

try:
    with mp_holistic.Holistic(min_detection_confidence=0.1, min_tracking_confidence=0.1) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = holistic.process(frame_rgb)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

                # Extract landmark coordinates (x, y, z for each keypoint)
                landmarks = results.pose_landmarks.landmark
                frame_landmarks = []
                for landmark in landmarks:
                    frame_landmarks.append({
                        'x': landmark.x,
                        'y': landmark.y,
                        'z': landmark.z,
                        'visibility': landmark.visibility
                    })

                # Append the frame's landmarks data to the list
                landmarks_data.append(frame_landmarks)

            # Encode and display the frame
            _, img = cv2.imencode('.png', frame)
            display(Image(data=img.tobytes()))
            clear_output(wait=True)
finally:
    cap.release()

# After processing the video, convert the landmarks data to a DataFrame
# Flatten the list of frames to make it suitable for a DataFrame
flat_landmarks_data = []
for frame_idx, frame_landmarks in enumerate(landmarks_data):
    for idx, landmark in enumerate(frame_landmarks):
        flat_landmarks_data.append({
            'frame': frame_idx,
            'landmark_idx': idx,
            'x': landmark['x'],
            'y': landmark['y'],
            'z': landmark['z'],
            'visibility': landmark['visibility']
        })

# Create the dataframe
df = pd.DataFrame(flat_landmarks_data)

# Display the dataframe
print(df)

       frame  landmark_idx         x         y         z  visibility
0          0             0  0.392972  0.287764 -0.326286    0.999803
1          0             1  0.395835  0.271543 -0.317401    0.999002
2          0             2  0.397460  0.270271 -0.317429    0.999003
3          0             3  0.399255  0.268972 -0.317506    0.998881
4          0             4  0.390587  0.271117 -0.319403    0.998921
...      ...           ...       ...       ...       ...         ...
25999    787            28  0.467406  1.147776  0.011501    0.784446
26000    787            29  0.342795  1.121633  0.160955    0.724378
26001    787            30  0.454737  1.181530  0.014894    0.801034
26002    787            31  0.382906  1.126658  0.088427    0.555586
26003    787            32  0.509656  1.198538 -0.076506    0.664685

[26004 rows x 6 columns]


In [9]:
df.to_pickle("../data/raw_coordinates.pkl.xz")

In [1]:
import pandas as pd

df = pd.read_pickle("../data/raw_coordinates.pkl.xz")

In [2]:
l = []

for landmark_index in df.landmark_idx.unique():
    subset_df = df.loc[df.landmark_idx == landmark_index]

